In [ ]:
import os
import pandas as pd
import pickle
from collections import defaultdict

TRAIN_META_PATH = "/media/theruins/Data/MABe Challenge/data/MABe-mouse-behavior-detection/train.csv"
TRACKING_ROOT = "/media/theruins/Data/MABe Challenge/data/MABe-mouse-behavior-detection/train_tracking"
ANN_ROOT = "/media/theruins/Data/MABe Challenge/data/MABe-mouse-behavior-detection/train_annotation"

OUT_DATA_PATH = "DLC2A_data"
OUT_LABELS_PATH = "DLC2A_labels"

os.makedirs(OUT_DATA_PATH, exist_ok=True)
os.makedirs(OUT_LABELS_PATH, exist_ok=True)


def convert_one_video_to_dlc2action(lab_id: str, video_id: str):
    """
    Convert 1 video:
      - train_tracking/{lab_id}/{video_id}.parquet
      - train_annotation/{lab_id}/{video_id}.parquet

    -> nhiều file pose .h5 (1 file / mouse_id)
    -> nhiều file label _dlc.pickle (1 file / agent_id)
    """
    tracking_path = os.path.join(TRACKING_ROOT, lab_id, f"{video_id}.parquet")
    ann_path = os.path.join(ANN_ROOT, lab_id, f"{video_id}.parquet")

    print(f"\n====== CONVERT {lab_id}/{video_id} ======")
    print("  Tracking path:", tracking_path, "exists?", os.path.isfile(tracking_path))
    print("  Ann path     :", ann_path, "exists?", os.path.isfile(ann_path))

    if not os.path.isfile(tracking_path):
        print("[SKIP] Không tìm thấy tracking:", tracking_path)
        return False
    if not os.path.isfile(ann_path):
        print("[SKIP] Không tìm thấy annotation:", ann_path)
        return False

    # ---------- 1) TRACKING ----------
    track_df = pd.read_parquet(tracking_path)
    # track_df = track_df.rename(columns={"frame": "video_frame", ...})

    required_cols = {"video_frame", "mouse_id", "bodypart", "x", "y"}
    missing = required_cols - set(track_df.columns)
    if missing:
        raise ValueError(f"Tracking file {tracking_path} thiếu cột: {missing}")

    for mid, g in track_df.groupby("mouse_id"):
        g = g.copy().sort_values("video_frame")

        # pivot: index = frame, columns = ('x'/'y', bodypart)
        wide_xy = g.pivot_table(
            index="video_frame",
            columns="bodypart",
            values=["x", "y"],
            aggfunc="mean",
        ).sort_index()

        # MultiIndex 4 level: (scorer, individuals, bodyparts, coords)
        tuples_xy = []
        for coord, bp in wide_xy.columns.to_flat_index():  # coord: 'x'/'y'
            tuples_xy.append(("mars", "ind0", bp, coord))

        wide_xy.columns = pd.MultiIndex.from_tuples(
            tuples_xy,
            names=["scorer", "individuals", "bodyparts", "coords"],
        )

        bodyparts = wide_xy.columns.get_level_values("bodyparts").unique()
        lik_tuples = [("mars", "ind0", bp, "likelihood") for bp in bodyparts]
        lik_cols = pd.MultiIndex.from_tuples(
            lik_tuples,
            names=["scorer", "individuals", "bodyparts", "coords"],
        )
        wide_lik = pd.DataFrame(
            1.0, index=wide_xy.index, columns=lik_cols, dtype=float
        )

        wide = pd.concat([wide_xy, wide_lik], axis=1).sort_index(axis=1)

        clip_id = f"{lab_id}_{video_id}_mouse{int(mid)}"
        out_h5 = os.path.join(OUT_DATA_PATH, f"{clip_id}.h5")
        wide.to_hdf(out_h5, key="df_with_missing")
        print(f"  [OK] Saved pose: {out_h5}")

    # ---------- 2) ANNOTATION ----------
    ann_df = pd.read_parquet(ann_path)
    # ann_df = ann_df.rename(columns={"start": "start_frame", "end": "stop_frame", ...})

    required_cols_ann = {"agent_id", "action", "start_frame", "stop_frame"}
    missing_ann = required_cols_ann - set(ann_df.columns)
    if missing_ann:
        raise ValueError(f"Annotation file {ann_path} thiếu cột: {missing_ann}")

    for aid, g in ann_df.groupby("agent_id"):
        beh_dict = defaultdict(list)
        for _, row in g.iterrows():
            beh = row["action"]
            s = int(row["start_frame"])
            e = int(row["stop_frame"])
            beh_dict[beh].append([s, e])

        dlc_dict = {"ind0": dict(beh_dict)}

        clip_id = f"{lab_id}_{video_id}_mouse{int(aid)}"
        out_pkl = os.path.join(OUT_LABELS_PATH, f"{clip_id}_dlc.pickle")
        with open(out_pkl, "wb") as f:
            pickle.dump(dlc_dict, f)
        print(f"  [OK] Saved label: {out_pkl}")

    print(f"[DONE] {lab_id}/{video_id}")
    return True


In [ ]:
def convert_all_from_meta(
    train_csv_path=TRAIN_META_PATH,
    labs_keep=("CRIM13", "CalMS21"),
):
    meta = pd.read_csv(train_csv_path)

    meta["lab_id"] = meta["lab_id"].astype(str).str.strip()
    meta["video_id"] = meta["video_id"].astype(str).str.strip()

    print("Các lab_id có trong train.csv:", meta["lab_id"].unique())

    if labs_keep is not None:
        labs_keep = {lab.strip() for lab in labs_keep}
        meta = meta[meta["lab_id"].isin(labs_keep)].copy()
        print("\nSau khi lọc labs_keep =", labs_keep,
              "=> số dòng còn lại:", len(meta))

    pairs = sorted(set(zip(meta["lab_id"], meta["video_id"])))
    print("Số (lab_id, video_id) sẽ convert:", len(pairs))

    n_ok, n_fail = 0, 0
    for lab_id, video_id in pairs:
        try:
            ok = convert_one_video_to_dlc2action(lab_id, video_id)
            if ok:
                n_ok += 1
            else:
                n_fail += 1
        except Exception as e:
            print(f"[ERROR] {lab_id}/{video_id}: {e}")
            n_fail += 1

    print(f"\nTỔNG KẾT: OK = {n_ok}, FAIL = {n_fail}")

convert_all_from_meta(labs_keep=("CRIM13",))

Các lab_id có trong train.csv: ['AdaptableSnail' 'BoisterousParrot' 'CRIM13' 'CalMS21_supplemental'
 'CalMS21_task1' 'CalMS21_task2' 'CautiousGiraffe' 'DeliriousFly'
 'ElegantMink' 'GroovyShrew' 'InvincibleJellyfish' 'JovialSwallow'
 'LyricalHare' 'MABe22_keypoints' 'MABe22_movies' 'NiftyGoldfinch'
 'PleasantMeerkat' 'ReflectiveManatee' 'SparklingTapir' 'TranquilPanther'
 'UppityFerret']

Sau khi lọc labs_keep = {'CRIM13'} => số dòng còn lại: 21
Số (lab_id, video_id) sẽ convert: 21

====== CONVERT CRIM13/1009459450 ======
  Tracking path: /media/theruins/Data/MABe Challenge/data/MABe-mouse-behavior-detection/train_tracking/CRIM13/1009459450.parquet exists? True
  Ann path     : /media/theruins/Data/MABe Challenge/data/MABe-mouse-behavior-detection/train_annotation/CRIM13/1009459450.parquet exists? True
  [OK] Saved pose: DLC2A_data/CRIM13_1009459450_mouse1.h5
  [OK] Saved pose: DLC2A_data/CRIM13_1009459450_mouse2.h5
  [OK] Saved label: DLC2A_labels/CRIM13_1009459450_mouse1_dlc.pickle
[

In [3]:
from dlc2action.project import Project
import os

CURRENT_PATH = os.getcwd()
DATA_PATH = os.path.join(CURRENT_PATH, "DLC2A_data")
LABELS_PATH = os.path.join(CURRENT_PATH, "DLC2A_labels")
PROJECTS_PATH = os.path.join(CURRENT_PATH, "DLC2Action_projects")

project = Project(
    "mabe_crim13",
    data_path=DATA_PATH,
    annotation_path=LABELS_PATH,
    projects_path=PROJECTS_PATH,
    data_type="dlc_track",
    annotation_type="dlc",
)



/media/theruins/Data/MABe Challenge/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import glob
import pickle
# Lấy list behaviors từ pickle
behaviors_set = set()
for pkl_path in glob.glob(os.path.join(LABELS_PATH, "*_dlc.pickle")):
    with open(pkl_path, "rb") as f:
        d = pickle.load(f)
    behaviors_set.update(d["ind0"].keys())
behaviors = sorted(behaviors_set)
print("Behaviors:", behaviors)

num_classes = len(behaviors) + 1


Behaviors: ['approach', 'attack', 'disengage', 'mount', 'rear', 'selfgroom', 'sniff']


In [ ]:
project.update_parameters(
    {
        "general": {
            "exclusive": True,                  # mỗi frame 1 nhãn
            "model_name": "c2f_tcn",
            "metric_functions": ["f1", "precision"],
            "ignored_clips": [],
            "len_segment": 512,
            "overlap": 0.75,                    # tỉ lệ overlap
            "interactive": False,               # per-mouse
            "num_classes": num_classes,
        },

        "data": {
            "data_suffix": ".h5",               # CRIM13_video_x_mousey.h5
            "feature_suffix": None,
            "annotation_suffix": "_dlc.pickle", # *_dlc.pickle
            "canvas_shape": [640, 480],
            "fps": 25.0,
            "ignored_bodyparts": [],
            "likelihood_threshold": 0.0,
            "behaviors": behaviors,
            "filter_annotated": True,
            "filter_background": True,
            "visibility_min_score": 0.0,
            "visibility_min_frac": 0.0,
        },

        "training": {
            "lr": 1e-3,
            "device": "auto",                   # tự dùng GPU nếu có
            "num_epochs": 50,                   # full train
            "to_ram": False,                    # bật True nếu RAM lớn
            "batch_size": 16,                   # tăng so với debug
            "normalize": True,
            "temporal_subsampling_size": 0.85,
            "parallel": False,
            "val_frac": 0.2,
            "test_frac": 0.0,
            "partition_method": "random",
        },

        "losses": {
            "ms_tcn": {
                "focal": True,
                "gamma": 2.0,
                "alpha": 0.001,
            },
        },

        "metrics": {
            "f1": {
                "average": "macro",
                "ignored_classes": None,
                "tag_average": "micro",
                "threshold_value": 0.5,
            },
            "precision": {
                "average": "macro",
                "ignored_classes": None,
                "tag_average": "micro",
                "threshold_value": 0.5,
            },
        },

        "model": {
            "num_f_maps": 128,
            "feature_dim": 128,
            "num_classes": num_classes,
        },

        "features": {
            # dùng toàn bộ kinematic mặc định
            "keys": None,
            "averaging_window": 1,
            "distance_pairs": None,
            "angle_pairs": None,
            "zone_vertices": None,
            "zone_bools": None,
            "zone_distances": None,
            "area_vertices": None,
        },

        "augmentations": {
            "augmentations": {"add_noise", "rotate", "shift"},
            "rotation_limits": [-0.7, 0.7],
            "mirror_dim": {0},
            "noise_std": 0.003,
            "zoom_limits": [0.8, 1.2],
            "masking_probability": 0.1,
        },
    },
)


In [8]:
episode_name = "tcn-c2f_"

project.run_episode(episode_name)


TRAINING tcn-c2f_
Computing input features...


100%|██████████| 21/21 [00:04<00:00,  4.69it/s]


Computing annotation arrays...


100%|██████████| 21/21 [00:00<00:00, 8694.15it/s]


Filtering 4.87% of samples
Number of samples:
    validation:
      ['-100: 54481', '0: 0', '1: 4916', '2: 1488', '3: 6035', '4: 4717', '5: 5976', '6: 8116', '7: 14623']
    training:
      ['-100: 371598', '0: 0', '1: 35114', '2: 26882', '3: 41789', '4: 36070', '5: 42192', '6: 49728', '7: 96531']
    test:
      ['0: 0', '1: 0', '2: 0', '3: 0', '4: 0', '5: 0', '6: 0', '7: 0']
Computing normalization statistics...


100%|██████████| 1367/1367 [00:01<00:00, 1191.85it/s]


Initializing class weights:
    13670000000.0, 0.039, 0.051, 0.033, 0.038, 0.032, 0.027, 0.014
Behavior indices:
    0: other
    1: approach
    2: attack
    3: disengage
    4: mount
    5: rear
    6: selfgroom
    7: sniff
[epoch 1]: loss 0.0281, f1 0.307, precision 0.313
validation: loss 0.0308, f1 0.172, precision 0.203
[epoch 2]: loss 0.0204, f1 0.390, precision 0.392
validation: loss 0.0235, f1 0.292, precision 0.345
[epoch 3]: loss 0.0178, f1 0.437, precision 0.437
validation: loss 0.0259, f1 0.268, precision 0.387
[epoch 4]: loss 0.0166, f1 0.465, precision 0.462
validation: loss 0.0210, f1 0.352, precision 0.413
[epoch 5]: loss 0.0153, f1 0.493, precision 0.487
validation: loss 0.0210, f1 0.352, precision 0.388
[epoch 6]: loss 0.0134, f1 0.551, precision 0.543
validation: loss 0.0297, f1 0.361, precision 0.404
[epoch 7]: loss 0.0132, f1 0.559, precision 0.549
validation: loss 0.0222, f1 0.376, precision 0.408
[epoch 8]: loss 0.0113, f1 0.598, precision 0.586
validation: los

In [10]:
results = project.evaluate([episode_name])
print(type(results))
print(results.keys())

EVALUATION tcn-c2f_
episode tcn-c2f_
Number of samples:
    validation:
      ['-100: 54481', '0: 0', '1: 4916', '2: 1488', '3: 6035', '4: 4717', '5: 5976', '6: 8116', '7: 14623']
    training:
      ['-100: 371598', '0: 0', '1: 35114', '2: 26882', '3: 41789', '4: 36070', '5: 42192', '6: 49728', '7: 96531']
    test:
      ['0: 0', '1: 0', '2: 0', '3: 0', '4: 0', '5: 0', '6: 0', '7: 0']
Setting loaded normalization statistics...
Initializing class weights:
    13670000000.0, 0.039, 0.051, 0.033, 0.038, 0.032, 0.027, 0.014
Behavior indices:
    0: other
    1: approach
    2: attack
    3: disengage
    4: mount
    5: rear
    6: selfgroom
    7: sniff


100%|██████████| 13/13 [00:00<00:00, 24.46it/s]




EVALUATE PREDICTION:
Inference time: 0:00:00


<class 'dict'>
dict_keys(['f1', 'precision'])


In [11]:
f1_res = results["f1"]

print(f1_res)

0.5211392045021057
